In [4]:
from commons import Conv2dSame
import torch
import numpy as np
import tensorflow as tf

amp_factor = 2

input_shape = (672, 100, 3, 1)

# Create the constant tensor
random_array = np.random.rand(672, 100, 3, 1)
x = tf.convert_to_tensor(random_array, dtype=tf.float32)

conv1 = tf.layers.conv2d(x, filters=32*amp_factor, kernel_size=[5, 3],
                                 data_format='channels_last', padding= "same",
                                 strides=(2, 1),
                                 )

m = Conv2dSame(in_channels=1, out_channels=32*amp_factor, kernel_size=(5, 3), stride=(2, 1))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output  = sess.run(conv1)
    output_transposed = np.transpose(output, (0, 3, 1, 2))

    vars = sess.graph.get_collection('trainable_variables')
    weights = {}
   
    for v in vars[-2:]:
        weights[v.name] = sess.run(v)

    # print(weights)

    with torch.no_grad():
        for (name, param), (tf_name, tf_param) in zip(m.named_parameters(), weights.items()):
            # convert NHWC to NCHW format and copy to change memory layout
            tf_param = np.transpose(tf_param, (3, 2, 0, 1)).copy() if len(tf_param.shape) == 4 else tf_param
            assert tf_param.shape == param.detach().numpy().shape, name

            # https://discuss.pytorch.org/t/how-to-assign-an-arbitrary-tensor-to-models-parameter/44082/3
            param.copy_(torch.tensor(tf_param, requires_grad=True, dtype=param.dtype))
    x_list = x.eval()
    x_np= np.transpose(x_list, (0, 3, 1, 2)).copy()
    x_pt = torch.tensor(x_np, dtype=torch.float32)
    output_pt = m(x_pt)
    output_pt_np = output_pt.detach().numpy()
    print("Tensorflow", output_transposed)
    print("Pytorch", output_pt_np)
    print(np.abs(output_transposed-output_pt_np).max())

    print(np.allclose(output_transposed, output_pt_np))


Tensorflow [[[[-6.35924935e-02  1.19365349e-01 -4.99760471e-02]
   [-9.18787941e-02  1.99113756e-01 -4.42330465e-02]
   [-4.76218797e-02  1.29426688e-01  3.22588114e-03]
   ...
   [-2.92713232e-02  1.21532895e-01  2.30572000e-02]
   [-1.00898957e-02  6.17750660e-02  2.31592674e-02]
   [-5.15559129e-03 -8.99351109e-03  1.63322571e-03]]

  [[ 9.76558253e-02  4.74079698e-02  7.31575713e-02]
   [ 8.65020901e-02 -1.93846337e-02  5.17262071e-02]
   [ 1.25846639e-01 -4.21848558e-02  6.80399761e-02]
   ...
   [ 1.11611739e-01  8.64829123e-03  3.69574055e-02]
   [ 7.19345585e-02  1.06015541e-01  2.68663391e-02]
   [ 4.90899198e-03  4.54308316e-02 -1.98042076e-02]]

  [[-3.26835066e-02  1.89461838e-02 -1.10550858e-01]
   [-1.01775102e-01 -2.67556198e-02 -9.81316417e-02]
   [-1.11283295e-01 -4.30755392e-02 -5.12542240e-02]
   ...
   [-1.61520764e-01 -7.68236443e-02 -1.34407476e-01]
   [-9.57303941e-02 -4.78741154e-02 -1.31871283e-01]
   [-3.50645110e-02 -8.46682638e-02 -1.13605678e-01]]

  ...

 

ValueError: operands could not be broadcast together with shapes (672,50,3,64) (672,64,50,3) 